In [33]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List, Annotated
from langchain_core.messages import HumanMessage, BaseMessage 
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langgraph.checkpoint.memory import MemorySaver

In [34]:
from langgraph.graph.message import add_messages
class chatState(TypedDict):
    messages : Annotated[List[BaseMessage], add_messages]

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
def chat_node(state:chatState):
    # take user query from state
    messages = state["messages"]
    # send it to llm
    response = llm.invoke(messages)
    # store the response in state
    return {"messages": [response]}

In [36]:
checkpointer = MemorySaver()
graph = StateGraph(chatState)
graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)
workflow = graph.compile(checkpointer=checkpointer)

In [40]:
thread_id = 1
while True:
    query = input("Type here: ").strip().lower()
    print("You: ", query)
    if query in ['exit', 'quit', 'bye']:
        break
    config = {"configurable": {"thread_id": thread_id}}
    response = workflow.invoke({"messages" : [HumanMessage(content=query)]}, config=config)
    print("AI: ", response["messages"][-1].content)

You:  hello i am satvik
AI:  Hello again, Satvik!

It's good to hear from you. What's on your mind?
You:  what's 10+20
AI:  10 + 20 = 30
You:  add 30 to the result
AI:  Okay, the previous result was 30.

Adding 30 to that: 30 + 30 = **60**
You:  bye bye
AI:  Goodbye, Satvik! Have a great day!
You:  bye
